In [1]:
from sentimentanalysis import *

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from time import time
from nltk import SnowballStemmer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

Loading dataset
0 documents
nan


/home/thomas/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/thomas/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Question 1

In [2]:
## On charges les 2000 documents

In [3]:
print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('34data', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('34data', 'data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

Loading dataset
2000 documents


In [4]:
voc, table = count_words(texts)
len(voc)

50920

## Question 2

## Question 3

## Utilisation de Sickit Learn

## Question 1

### transformation de nos données pour effectuer les modèles et création d'un dataset train et test

In [5]:
corpus = texts
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(len(vectorizer.get_feature_names()))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

39659


In [6]:
def ErrorRate(Y_estim, Y):
    '''Fonction qui calcule le taux d\'erreur '''
    Y_error = Y_estim - Y
    Error_Rate = len(Y_error[Y_error!=0]) / len(Y_error)
    print('le taux d\'erreur sur les données de test vaut', round(Error_Rate,3))

In [7]:
Nb = MultinomialNB()
Nb.fit(X_train, y_train)
ErrorRate(Nb.predict(X_test), y_test)


le taux d'erreur sur les données de test vaut 0.213


### création d'une pipeline avec GridSearch pour le Naives Bayes

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (2,2)) ),
    ('clf', MultinomialNB()),
    ])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),       # ignore terms that have a document frequency strictly higher than the given threshold
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__alpha': (0.1,1,2,3),                # additive (Laplace/Lidstone) smoothing parameter
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, verbose=0, n_jobs=-1)

print("Performing grid search...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
pprint(parameters)
t0 = time()
grid_search.fit(texts, y)
print("done in %0.3fs" % (time() - t0))
print()
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline : ['vect', 'clf']
parameters :
{'clf__alpha': (0.1, 1, 2, 3),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
done in 144.612s

Best score: 0.838
Best parameters set :
	clf__alpha: 1
	vect__max_df: 0.75
	vect__ngram_range: (1, 2)


## Question 2

### Pipeline pour tester plusieurs modèles

In [9]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

multiNB_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2)) ),
    ('multiNB', MultinomialNB() ),
    ])

svc_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2)) ),
    ('svc', LinearSVC(penalty='l2', random_state=2) ),
    ])

lr_pipe = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2)) ),
    ('lr', LogisticRegression(random_state=2) ),
    ])

parameters= {
    'vect__max_df':(0.67, 0.7, 0.72),
#    'vect__ngram_range':((1, 1),(1, 2)),

#    'multiNB__alpha' : (0.8, 1),
#    'svc__loss': ('squared_hinge', 'hinge'),

    'lr__penalty': ('l1','l2'),
}


pipeline = lr_pipe

grid_search = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=1, n_jobs=-1)

print("Performing grid search...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
pprint(parameters)
t0 = time()
grid_search.fit(texts, y)
print("done in %0.3fs" % (time() - t0))
print()
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline : ['vect', 'lr']
parameters :
{'lr__penalty': ('l1', 'l2'), 'vect__max_df': (0.67, 0.7, 0.72)}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.3min finished


done in 86.349s

Best score: 0.854
Best parameters set :
	lr__penalty: 'l2'
	vect__max_df: 0.67


## Question 3

In [10]:
# permet de remplacer les mots par leur racinisation
stemmer = SnowballStemmer("english")
test = texts
all_list = []

for iloop in range(len(test)):
    new_words_list = []
    for rep1 in test[iloop].split():
        add = stemmer.stem(rep1.split(' ')[0])
        new_words_list.append(add)
        
    all_list.append(' '.join(new_words_list))
        

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (2,2)) ),
    ('clf', MultinomialNB()),
    ])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),       # ignore terms that have a document frequency strictly higher than the given threshold
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__alpha': (0.1,1,2,3),                # additive (Laplace/Lidstone) smoothing parameter
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, verbose=0, n_jobs=-1)

print("Performing grid search...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
pprint(parameters)
t0 = time()
grid_search.fit(all_list, y)
print("done in %0.3fs" % (time() - t0))
print()
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline : ['vect', 'clf']
parameters :
{'clf__alpha': (0.1, 1, 2, 3),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
done in 142.614s

Best score: 0.833
Best parameters set :
	clf__alpha: 1
	vect__max_df: 0.5
	vect__ngram_range: (1, 2)


## Question 4

In [12]:
#sous linux aller dans le terminal est insérer
# python -m nltk.downloader 'punkt'
# python -m nltk.downloader 'averaged_perceptron_tagger'

# ou 
#import nltk as nltk
#nltk.download()


In [13]:
TestPo = texts
Token_list = []

for iloop in range(len(TestPo)):
    Token_list.append(word_tokenize(TestPo[iloop]))

In [14]:
Text_pos_tag = [pos_tag(sent) for sent in Token_list]


In [15]:
# on ne garde que les mots qui sont des noms, adjectifs, adverbes ou des verbes
grammar = 'NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBP','VBZ','JJ','JJR','JJS','RB','RBR','RBS'
final_list = []
for iloop in range(len(Text_pos_tag)):
    good_words_list = []
    for jloop in range(len(Text_pos_tag[iloop])):
        if Text_pos_tag[iloop][jloop][1] in grammar:
            good_words_list.append(Text_pos_tag[iloop][jloop][0])
    
    final_list.append(' '.join(good_words_list))
    

In [16]:
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (2,2)) ),
    ('clf', MultinomialNB()),
    ])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),       # ignore terms that have a document frequency strictly higher than the given threshold
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'clf__alpha': (0.1,1,2,3),                # additive (Laplace/Lidstone) smoothing parameter
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, verbose=0, n_jobs=-1)

print("Performing grid search...")
print("pipeline :", [name for name, _ in pipeline.steps])
print("parameters :")
pprint(parameters)
t0 = time()
grid_search.fit(final_list, y)
print("done in %0.3fs" % (time() - t0))
print()
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set :")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline : ['vect', 'clf']
parameters :
{'clf__alpha': (0.1, 1, 2, 3),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__ngram_range': ((1, 1), (1, 2))}
done in 108.255s

Best score: 0.828
Best parameters set :
	clf__alpha: 1
	vect__max_df: 0.75
	vect__ngram_range: (1, 2)
